# 📖 NCTB Class 3 Bengali Book — OCR Pipeline
### EasyOCR দিয়ে Scanned Bengali PDF → Unicode Text

**কেন EasyOCR?**
- NCTB Class 3 বাংলা বইটি scanned image PDF
- Standard Tesseract Bengali Unicode output দিতে পারে না এই ধরনের PDF এ
- EasyOCR deep learning based — বাংলার জন্য অনেক বেশি accurate

**Steps:**
1. PDF → Page images (pdftoppm)
2. EasyOCR দিয়ে Bengali text extract
3. Clean ও chunk করা
4. FAISS vector index তৈরি
5. Test retrieval

---
⚠️ **Setup:** PDF টা Kaggle Dataset হিসেবে add করো।  
Path হবে: `/kaggle/input/nctb-class3/Class-3__Bangla_combine__compressed.pdf`

## Step 1: Install Libraries

In [ ]:
# Core installs
!pip install -q easyocr
!pip install -q langchain langchain-community
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!apt-get install -q poppler-utils   # pdftoppm এর জন্য

print('✅ সব install হয়েছে!')

## Step 2: Setup ও PDF Check

In [ ]:
import os, subprocess, json, pickle
import numpy as np
from pathlib import Path

# ── Paths ──────────────────────────────────────────────────────
PDF_PATH  = '/kaggle/input/nctb-class3/Class-3__Bangla_combine__compressed.pdf'
WORK_DIR  = '/kaggle/working'
IMG_DIR   = f'{WORK_DIR}/pages'      # PDF pages as PNG
TEXT_DIR  = f'{WORK_DIR}/texts'      # Extracted text per page
INDEX_DIR = f'{WORK_DIR}/faiss_index'  # Final FAISS index

for d in [IMG_DIR, TEXT_DIR, INDEX_DIR]:
    os.makedirs(d, exist_ok=True)

# PDF exists check
if not os.path.exists(PDF_PATH):
    # Kaggle input এ না পেলে working directory তে খোঁজো
    alt = '/kaggle/working/Class-3__Bangla_combine__compressed.pdf'
    if os.path.exists(alt):
        PDF_PATH = alt
    else:
        raise FileNotFoundError(
            f'PDF পাওয়া যায়নি!\n'
            f'Kaggle Dataset এ PDF add করো এবং path ঠিক করো।\n'
            f'Expected: {PDF_PATH}'
        )

# Page count
result = subprocess.run(
    ['pdfinfo', PDF_PATH], capture_output=True, text=True
)
for line in result.stdout.split('\n'):
    if 'Pages' in line:
        TOTAL_PAGES = int(line.split(':')[1].strip())
        break

print(f'✅ PDF found: {PDF_PATH}')
print(f'📄 মোট পৃষ্ঠা: {TOTAL_PAGES}')
print(f'🗂️  Working dir: {WORK_DIR}')

## Step 3: PDF → Page Images (High DPI)

In [ ]:
from tqdm import tqdm

# ── Config ────────────────────────────────────────────────────
DPI         = 300    # High DPI = better OCR accuracy
START_PAGE  = 4      # প্রথম ৩ পৃষ্ঠা cover/blank, skip করি
END_PAGE    = TOTAL_PAGES
BATCH_SIZE  = 10     # কতটা page একসাথে convert করবে

def convert_batch(start, end, dpi=DPI):
    """PDF এর start থেকে end পৃষ্ঠা PNG তে convert করা।"""
    result = subprocess.run([
        'pdftoppm', '-png', '-r', str(dpi),
        '-f', str(start), '-l', str(end),
        PDF_PATH, f'{IMG_DIR}/page'
    ], capture_output=True)
    return result.returncode == 0

# Convert all pages in batches
print(f'🔄 Converting pages {START_PAGE}–{END_PAGE} to PNG @ {DPI} DPI...')
print(f'   (এটা ৫-১০ মিনিট সময় নিতে পারে)')

batches = range(START_PAGE, END_PAGE + 1, BATCH_SIZE)
for batch_start in tqdm(batches, desc='Converting batches'):
    batch_end = min(batch_start + BATCH_SIZE - 1, END_PAGE)
    convert_batch(batch_start, batch_end)

# Count converted images
pages = sorted(Path(IMG_DIR).glob('*.png'))
print(f'\n✅ {len(pages)} টি page image তৈরি হয়েছে।')
print(f'   Sample: {pages[0].name} ... {pages[-1].name}')

## Step 4: EasyOCR Reader Initialize

In [ ]:
import easyocr
import torch

print(f'GPU available: {torch.cuda.is_available()}')
print('🔄 EasyOCR model load হচ্ছে... (প্রথমবার একটু সময় লাগবে)')

# Bengali + English দুটোই recognize করবে
# gpu=True → Kaggle T4 GPU ব্যবহার করবে (অনেক faster)
READER = easyocr.Reader(
    ['bn', 'en'],           # Bengali + English
    gpu=torch.cuda.is_available(),
    verbose=False
)

print('✅ EasyOCR ready!')

# Quick test on first content page
test_img = str(pages[0])
test_result = READER.readtext(test_img, detail=0, paragraph=True)
print(f'\n🧪 Quick test on {pages[0].name}:')
print('\n'.join(test_result[:5]))

## Step 5: Full Book OCR

পুরো বইয়ের সব পৃষ্ঠা OCR করা হবে।  
প্রতিটি পৃষ্ঠার text আলাদা file এ save হবে — resume করতে সুবিধা হবে।

In [ ]:
import unicodedata
import re

def clean_bengali(text):
    """OCR output clean করা।"""
    text = unicodedata.normalize('NFC', text)  # Unicode normalize
    text = re.sub(r'\n{3,}', '\n\n', text)     # Extra blank lines সরানো
    text = re.sub(r'[ \t]+', ' ', text)         # Extra spaces সরানো
    return text.strip()

def ocr_page(img_path):
    """একটি page এর OCR করা।"""
    results = READER.readtext(
        str(img_path),
        detail=0,
        paragraph=True,       # Text কে paragraphs এ group করে
        width_ths=0.7,        # Word grouping threshold
        decoder='greedy'
    )
    return clean_bengali('\n'.join(results))

# ── Run OCR on all pages ───────────────────────────────────────
all_page_texts = {}  # {page_num: text}
failed_pages = []

print(f'🔄 {len(pages)} পৃষ্ঠার OCR শুরু হচ্ছে...')
print('   (প্রতি পৃষ্ঠায় ~৩-৫ সেকেন্ড, মোট ~১০ মিনিট)')
print('   Text file গুলো automatically save হতে থাকবে।\n')

for img_path in tqdm(pages, desc='OCR Progress'):
    # Page number extract করা filename থেকে
    page_num = int(img_path.stem.split('-')[-1])
    text_file = Path(TEXT_DIR) / f'page_{page_num:03d}.txt'
    
    # Already processed? Skip (resume support)
    if text_file.exists():
        with open(text_file, encoding='utf-8') as f:
            all_page_texts[page_num] = f.read()
        continue
    
    try:
        text = ocr_page(img_path)
        all_page_texts[page_num] = text
        
        # Save immediately (session crash হলেও progress নষ্ট হবে না)
        with open(text_file, 'w', encoding='utf-8') as f:
            f.write(text)
            
    except Exception as e:
        print(f'\n⚠️ Page {page_num} failed: {e}')
        failed_pages.append(page_num)
        all_page_texts[page_num] = ''

# Summary
total_words = sum(len(t.split()) for t in all_page_texts.values())
bengali_chars = sum(
    sum(1 for c in t if '\u0980' <= c <= '\u09FF')
    for t in all_page_texts.values()
)

print(f'\n✅ OCR Complete!')
print(f'   Pages processed: {len(all_page_texts)}')
print(f'   Total words: {total_words:,}')
print(f'   Bengali Unicode chars: {bengali_chars:,}')
if failed_pages:
    print(f'   ⚠️ Failed pages: {failed_pages}')

## Step 6: Sample দেখা

In [ ]:
# কয়েকটা page এর text দেখো
print('=' * 60)
print('📄 SAMPLE OUTPUT — OCR করা Bengali Text:')
print('=' * 60)

# First few non-empty pages
shown = 0
for pg, text in sorted(all_page_texts.items()):
    if text.strip() and len(text.split()) > 10:
        print(f'\n[পৃষ্ঠা {pg}]')
        print(text[:400])
        print('---')
        shown += 1
        if shown >= 3:
            break

# Bengali word count check
sample_text = list(all_page_texts.values())[5]
bn_chars = sum(1 for c in sample_text if '\u0980' <= c <= '\u09FF')
total_chars = len(sample_text)
print(f'\n📊 Bengali char ratio: {bn_chars}/{total_chars} = {bn_chars/max(total_chars,1)*100:.1f}%')

if bn_chars / max(total_chars, 1) > 0.3:
    print('✅ Bengali text ঠিকমতো extract হয়েছে!')
else:
    print('⚠️ Bengali chars কম — OCR quality check করো')

## Step 7: Full Book Text তৈরি + Chunking

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# সব page text একসাথে জোড়া লাগানো
FULL_TEXT = ''
for pg_num in sorted(all_page_texts.keys()):
    text = all_page_texts[pg_num]
    if text.strip():
        FULL_TEXT += f'\n\n--- পৃষ্ঠা {pg_num} ---\n{text}'

# Full text save করো
with open(f'{WORK_DIR}/class3_bangla_full.txt', 'w', encoding='utf-8') as f:
    f.write(FULL_TEXT)
print(f'✅ Full text saved: {len(FULL_TEXT):,} characters, {len(FULL_TEXT.split()):,} words')

# ── Chunking ──────────────────────────────────────────────────
# Bengali-aware: '।' (দাড়ি) কে separator হিসেবে ব্যবহার করা হয়েছে
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=75,
    separators=['\n\n', '\n', '।', '.', ' ', '']
)

CHUNKS = splitter.split_text(FULL_TEXT)

# Empty chunks filter করো
CHUNKS = [c.strip() for c in CHUNKS if len(c.strip().split()) >= 5]

print(f'📦 মোট chunks: {len(CHUNKS)}')
print(f'   Average chunk size: {sum(len(c) for c in CHUNKS) // len(CHUNKS)} chars')
print(f'\n🔍 Sample chunk:')
print(CHUNKS[10] if len(CHUNKS) > 10 else CHUNKS[0])

## Step 8: Embeddings + FAISS Index

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import torch

print('🔄 Embedding model load হচ্ছে...')

# Multilingual model — Bengali, English উভয়ই support করে
EMBED_MODEL = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

print(f'✅ Model loaded on: {"GPU" if torch.cuda.is_available() else "CPU"}')
print(f'🔄 {len(CHUNKS)} chunks embed করা হচ্ছে...')

EMBEDDINGS = EMBED_MODEL.encode(
    CHUNKS,
    batch_size=128,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print(f'\n✅ Embeddings shape: {EMBEDDINGS.shape}')

# ── FAISS Index ───────────────────────────────────────────────
dim = EMBEDDINGS.shape[1]  # 384
INDEX = faiss.IndexFlatIP(dim)  # Inner Product = Cosine similarity
INDEX.add(EMBEDDINGS.astype('float32'))

# Save everything
faiss.write_index(INDEX, f'{INDEX_DIR}/bangla_class3.faiss')
with open(f'{INDEX_DIR}/chunks.pkl', 'wb') as f:
    pickle.dump(CHUNKS, f)

print(f'\n💾 FAISS index saved!')
print(f'   Vectors stored: {INDEX.ntotal}')
print(f'   Files:')
print(f'   - {INDEX_DIR}/bangla_class3.faiss')
print(f'   - {INDEX_DIR}/chunks.pkl')

## Step 9: Retrieval Test

In [ ]:
def retrieve(question, k=3):
    """Student এর question এর জন্য NCTB content খুঁজে আনা।"""
    q_emb = EMBED_MODEL.encode(
        [question], normalize_embeddings=True, convert_to_numpy=True
    ).astype('float32')
    scores, indices = INDEX.search(q_emb, k)
    return [
        {'text': CHUNKS[i], 'score': float(s)}
        for s, i in zip(scores[0], indices[0])
        if i >= 0
    ]

# Test questions — Bengali ও English
TEST_QS = [
    'তোমার বাড়িতে কতজন আছে?',
    'বন্ধু মানে কী?',
    'What is this lesson about?',
    'পরিবারের সদস্য কারা?',
    'ধন্যবাদ কাকে বলি?'
]

print('=' * 60)
print('🧪 RETRIEVAL TEST')
print('=' * 60)

for q in TEST_QS:
    print(f'\n❓ প্রশ্ন: {q}')
    results = retrieve(q, k=2)
    for i, r in enumerate(results, 1):
        preview = r['text'][:200].replace('\n', ' ')
        print(f'   [{i}] Score={r["score"]:.3f}: {preview}...')
    print('-' * 40)

## Step 10: Phi-3 RAG Prompt Template

In [ ]:
def build_rag_prompt(student_question, k=3, mode='chat'):
    """
    Phi-3 এর জন্য complete RAG prompt।
    mode: 'chat' = answer দেওয়া, 'quiz' = MCQ generate করা
    """
    # Relevant context retrieve করো
    chunks = retrieve(student_question, k=k)
    context_text = '\n\n'.join(
        f'[অংশ {i+1}]:\n{c["text"]}' for i, c in enumerate(chunks)
    )
    
    if mode == 'chat':
        system = (
            'তুমি একটি AI শিক্ষক। বাংলাদেশের তৃতীয় শ্রেণির ছাত্রছাত্রীদের সাহায্য করো।\n'
            'শুধুমাত্র নিচের NCTB পাঠ্যবইয়ের অংশ ব্যবহার করে সহজ বাংলায় উত্তর দাও।\n'
            'উত্তর ছোট ও বন্ধুত্বপূর্ণ রাখো যেন একটি ৮-৯ বছরের বাচ্চা বুঝতে পারে।'
        )
        user = f'পাঠ্যবইয়ের অংশ:\n{context_text}\n\nছাত্রের প্রশ্ন: {student_question}'
        
    elif mode == 'quiz':
        system = (
            'তুমি একজন শিক্ষক। NCTB পাঠ্যবইয়ের তথ্য থেকে MCQ তৈরি করো।\n'
            'Output শুধু JSON format এ দাও।'
        )
        user = (
            f'পাঠ্যবইয়ের অংশ:\n{context_text}\n\n'
            f'"{student_question}" বিষয়ে ৩টি সহজ MCQ প্রশ্ন তৈরি করো।\n'
            'Format: {"questions": [{"q": "...", "options": ["ক) ...", "খ) ...", "গ) ...", "ঘ) ..."], "answer": "ক"}]}'
        )
    
    # Phi-3 prompt format
    prompt = f'<|system|>\n{system}\n<|end|>\n<|user|>\n{user}\n<|end|>\n<|assistant|>'
    return prompt

# Demo
sample_prompt = build_rag_prompt('বন্ধুত্ব মানে কী?', mode='chat')
print('📝 Sample Chat Prompt (Phi-3 ready):')
print('=' * 60)
print(sample_prompt[:800])
print('...')
print('=' * 60)

quiz_prompt = build_rag_prompt('পরিবার', mode='quiz')
print('\n📝 Sample Quiz Prompt:')
print('=' * 60)
print(quiz_prompt[:600])
print('...')

## ✅ Summary

In [ ]:
import os

faiss_size = os.path.getsize(f'{INDEX_DIR}/bangla_class3.faiss') / 1024
chunks_size = os.path.getsize(f'{INDEX_DIR}/chunks.pkl') / 1024

print('=' * 60)
print('✅ NCTB Class 3 Bengali — RAG Pipeline Complete!')
print('=' * 60)
print(f'''
📊 Stats:
   Total pages OCR'd : {len(all_page_texts)}
   Total words       : {len(FULL_TEXT.split()):,}
   Total chunks      : {len(CHUNKS)}
   FAISS vectors     : {INDEX.ntotal}
   Embedding dim     : {EMBEDDINGS.shape[1]}

💾 Saved Files:
   {WORK_DIR}/class3_bangla_full.txt  ({os.path.getsize(f"{WORK_DIR}/class3_bangla_full.txt")//1024} KB)
   {INDEX_DIR}/bangla_class3.faiss    ({faiss_size:.0f} KB)
   {INDEX_DIR}/chunks.pkl             ({chunks_size:.0f} KB)
   {TEXT_DIR}/page_*.txt              (per-page backups)

🚀 পরবর্তী Notebook:
   → Phi-3 Mini inference (ollama / transformers)
   → Whisper STT: Bengali audio → text
   → Edge-TTS: text → Bengali voice
   → Django API এ integrate করা
''')
print('=' * 60)